# Group 7 - Too Hot to Handle

### Dharmik Bhayani
### Khushboo Modi
### Surbhi Gupta

# Project Overview

“People are suffering. People are dying. Entire ecosystems are collapsing. We are in the beginning of a mass extinction.” The interest in climate change has never been stronger or more important than it is today.

Any effort to combat climate change will be multi-faceted. And the best place to start would be at the local level. Almost half of the world's population lives in cities. Also historically, local governments have been more successful in dealing with climate control as environmental initiatives often face challenges at the federal and state levels.
So if we make people aware about how climate has changed locally, the chances of successful ventures for climate control will definitely increase. 

These are the questions which we seek to answer:
    1. How has precipitation changed over the years?
    2. How weather has changed over the last two decades?
    3. Compare differences in temperature and precipitation with respect to Geographical features.
    
To answer these questions we used data from "https://www.ncdc.noaa.gov/IPS/cd/cd.html". This site provides pdfs of monthly or annual weather report from all weather stations in the state, upon request. We decided to use Maryland, Virginia, Delaware and DC regions' annual weather reports from the year 1920 till 2018 for our analysis. The data was in the pdf in the form of tables. The tables had monthly Precipitation, Temperature, Cooling Days data from each weather station in that respective state.

# Step 1

# Data Acquisition and Cleaning 

We scraped pdfs for Maryland, Virginia, Delaware and DC for years 1920 till 2018 using Selenium from "https://www.ncdc.noaa.gov/IPS/cd/cd.html" and saved all the pdf data using requests library. <br>
To convert the pdfs into dataframes we used camelot.

## Scanning the internet for data 

In [1]:
#importing the packages
from selenium import webdriver  
from selenium.webdriver.support.ui import Select  
import time  
import requests  
import os  
import pandas as pd
import numpy as np
import camelot
import re
import json, string
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

from folium import Map, Marker, GeoJson, LayerControl

%matplotlib inline

In [2]:
#Opening a bot controlled browser
driver = webdriver.Firefox()

In [3]:
# Creating a list for file names
year_list = []  
md_file_name = []  
va_file_name = []  
for i in range(1920,2019):  
    
    pub_year = str(i)+'-ANNUAL'  
    
    year_list.append(pub_year)  
    
    cwd = os.getcwd()  
    va = '\\VA'  
    md = '\\MD'  
    md_file = '\\MD' + str(i) + '.pdf'  
    va_file = '\\VA' + str(i) + '.pdf'  
    md_link = cwd+md+md_file  
    va_link = cwd+va+va_file  
    
    md_file_name.append(md_link)  
    va_file_name.append(va_link)  

In [4]:
print("The Maryland list of file names and path looks like this: " + md_file_name[1])
print("The Virginia list of file names and path looks like this: " + va_file_name[1])

The Maryland list of file names and path looks like this: C:\Users\dharm\Jupyter Workspace\project\MD\MD1921.pdf
The Virginia list of file names and path looks like this: C:\Users\dharm\Jupyter Workspace\project\VA\VA1921.pdf


#### Going through the internet to get temporary links of the pdf generated on NOAA websites

In [5]:
#Getting the pdf links for Maryland
md_pdf_link = []  
for year in year_list:  
    
    driver.get("https://www.ncdc.noaa.gov/IPS/cd/cd.html")  
    state = Select(driver.find_element_by_name('state'))  
    state.select_by_value('MD')  
    next_btn = driver.find_element_by_name('_target1')  
    next_btn.click()  
    
    time.sleep(1)  
    
    publication = Select(driver.find_element_by_name('publicationDate'))  
    publication.select_by_value(year)  
    submit_btn = driver.find_element_by_name('_target2')  
    submit_btn.click()  
    
    time.sleep(2)  
    
    link = driver.find_element_by_id('anch_16')  
    pdf_link = link.get_attribute('href')  
    
    md_pdf_link.append(pdf_link)

print("Fetching of links for Maryland -- Completed")

Fetching of links for Maryland -- Completed


In [6]:
#Getting the pdf links for Virginia
va_pdf_link = []  
for year in year_list:  
    
    driver.get("https://www.ncdc.noaa.gov/IPS/cd/cd.html")  
    state = Select(driver.find_element_by_name('state'))  
    state.select_by_value('VA')  
    next_btn = driver.find_element_by_name('_target1')  
    next_btn.click()  
    
    time.sleep(1)  
    
    publication = Select(driver.find_element_by_name('publicationDate'))  
    publication.select_by_value(year)  
    submit_btn = driver.find_element_by_name('_target2')  
    submit_btn.click()  
    
    time.sleep(2)  
    
    link = driver.find_element_by_id('anch_16')  
    pdf_link = link.get_attribute('href')  
    
    va_pdf_link.append(pdf_link)
    
print("Fetching of links for Virginia -- Completed")

Fetching of links for Virginia -- Completed


#### Saving the pdf files to system

In [7]:
for i in range(99):  
    r = requests.get(md_pdf_link[i], stream = True)  

    with open(md_file_name[i],"wb") as pdf:   
        for chunk in r.iter_content(chunk_size=1024):   
            if chunk:   
                pdf.write(chunk)

print("Saving of Maryland files to system -- Completed")

Saving of Maryland files to system -- Completed


In [8]:
for i in range(99):  
    r = requests.get(va_pdf_link[i], stream = True)  

    with open(va_file_name[i],"wb") as pdf:   
        for chunk in r.iter_content(chunk_size=1024):   
            if chunk:   
                pdf.write(chunk)  
                
print("Saving of Virginia files to system -- Completed")

Saving of Virginia files to system -- Completed


In [9]:
#Closing the browser
driver.close()

# Step 2

# Scraping data from pdfs and Data Cleaning

Limitations: 
1. After downloading the pdf from the internet using selenium, we found out that annual reports from 1921 to 2004 are image based and we were unable to scrape data from that reports, hence we needed to limit the data cleaning from 2005-2018
2. As each pdf behaved and scraped in a different way, we were forced to clean each pdf manually and we were unable to automate the process of cleaning.
3. Initially we used the tabula-py package, that did not work hence we used camelot package to scrape the pdf.

### Maryland Rain Data

#### MD2018

In [10]:
#Setting the path to the correct directory
path_a = os.getcwd() + '\\MD\\'
path_a

'C:\\Users\\dharm\\Jupyter Workspace\\project\\MD\\'

In [11]:
#Setting the path to the required pdf and using Camelot to scrape specific pages from the file.
file = path_a + 'MD2018.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

<TableList n=2>

The report contains precipitation data on page 2,3 hence we scrape two pages which leads to two tables for precipitation.

In [12]:
try1 = tables[0].df
try1

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,STATION,JAN,,FEB,,MAR,,APR,,MAY,,JUN,
1,,PRECIP.,DEPART.,PRECIP.,DEPART.,PRECIP.,DEPART.,PRECIP.,DEPART.,PRECIP.,DEPART.,PRECIP.,DEPART.
2,MARYLAND\nSOUTHERN EASTERN SHORE 01\nPRINCESS...,M\n4.63\n3.53\n4.15\n4.10\n3.27\n3.27\n2.43\n ...,-0.08\n0.58\n0.10\n-0.33\n-0.71\n-0.80\n \n-0....,M\n2.88\n2.72\nM 1.90\n2.80\n5.93\n5.93\n6.73\...,-0.63\n-1.24\n-0.62\n2.80\n2.62\n3.65\n \n2.41...,M\nM\n3.04\n3.39\n3.22\n3.27\n3.27\n3.07\nMA 3...,-1.38\n-1.13\n-1.26\n-1.09\n-1.02\n-1.14\n \n-...,M\n3.08\n2.31\n2.36\n2.58\n3.30\n3.30\n3.68\n ...,-1.24\n-1.12\n-0.68\n-0.56\n-0.09\n0.18\n \n0....,M\n11.39\n9.79\nM\n10.59\n7.73\n7.73\n11.59\nM...,6.17\n \n6.97\n3.54\n3.82\n7.39\n \n7.45\n \n4...,M\n5.25\n4.17\n4.60\n4.67\n2.79\n2.79\n7.06\nM...,0.46\n1.13\n1.29\n-1.07\n-0.76\n2.99\n \n3.28\...


Each row in the table is seperated by \n, hence we split each column by \n and each entry in new row and then again combine all columns to make a dataframe. And filling blank values with mean of each column.

In [13]:
#Cleaning the first table on first page
try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)

In [14]:
#Cleaning the second table on second page
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)

In [15]:
#Combining the two tables into one dataframe for MD 2018 and cleaning the whole dataframe
MDrain2018 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2018.columns = ['Station','JAN_Precip_18','JAN_Depart_18','FEB_Precip_18','FEB_Depart_18','MAR_Precip_18','MAR_Depart_18','APR_Precip_18','APR_Depart_18','MAY_Precip_18','MAY_Depart_18','JUN_Precip_18','JUN_Depart_18','Station1','JUL_Precip_18','JUL_Depart_18','AUG_Precip_18','AUG_Depart_18','SEP_Precip_18','SEP_Depart_18','OCT_Precip_18','OCT_Depart_18','NOV_Precip_18','NOV_Depart_18','DEC_Precip_18','DEC_Depart_18']

MDrain2018 = MDrain2018.set_index('Station')
MDrain2018 = MDrain2018.drop(columns=['Station1'])

for x in MDrain2018.columns:  
    MDrain2018[x] = MDrain2018[x].str.replace(r'[A-Z]','')
    MDrain2018[x] = MDrain2018[x].str.replace('      ','0')
    MDrain2018[x] = MDrain2018[x].str.replace(' ','')
    MDrain2018[x] = MDrain2018[x].replace(r'', np.NaN)
    MDrain2018[x] = MDrain2018[x].astype('float')
MDrain2018 = MDrain2018.fillna(MDrain2018.mean())

print("MD Rain Data for 2018 cleaned and dataframe created")
print("")

MD Rain Data for 2018 cleaned and dataframe created



In [16]:
MDrain2018.head()

,JAN_Precip_18,JAN_Depart_18,FEB_Precip_18,FEB_Depart_18,MAR_Precip_18,MAR_Depart_18,APR_Precip_18,APR_Depart_18,MAY_Precip_18,MAY_Depart_18,...,AUG_Precip_18,AUG_Depart_18,SEP_Precip_18,SEP_Depart_18,OCT_Precip_18,OCT_Depart_18,NOV_Precip_18,NOV_Depart_18,DEC_Precip_18,DEC_Depart_18
Station,,,,,,,,,,,,,,,,,,,,,
MARYLAND,2.405625,-0.08,5.004063,-0.63,2.642667,-1.38,3.658387,-1.24,7.767931,6.170000,...,3.22,-1.440000,7.04,3.200000,6.80,3.500000,5.86,2.470,5.330345,-0.18
SOUTHERN EASTERN SHORE 01,4.630000,0.58,2.880000,-1.24,2.642667,-1.13,3.080000,-1.12,11.390000,3.658333,...,3.30,1.477308,7.73,5.664615,8.48,0.635385,6.39,3.734,4.670000,-0.38
PRINCESS ANNE,3.530000,0.10,2.720000,-0.62,3.040000,-1.26,2.310000,-0.68,9.790000,6.970000,...,2.26,-2.170000,5.26,1.280000,8.67,5.180000,4.50,1.070,3.530000,0.43
SALISBURY 2N,4.150000,-0.33,1.900000,2.80,3.390000,-1.09,2.360000,-0.56,7.767931,3.540000,...,2.63,-2.270000,7.47,3.070000,1.49,-2.040000,4.50,0.980,3.290000,0.87
SALISBURY-WICOMICO RGNL AP //,4.100000,-0.71,2.800000,2.62,3.220000,-1.02,2.580000,-0.09,10.590000,3.820000,...,2.85,-1.890000,6.88,3.120000,7.98,4.610000,5.13,1.950,3.830000,1.12


#### The same process is repeated for the years 2005 to 2017

In [17]:
#MD2017
file = path_a + 'MD2017.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2017 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2017.columns = ['Station','JAN_Precip_17','JAN_Depart_17','FEB_Precip_17','FEB_Depart_17','MAR_Precip_17','MAR_Depart_17','APR_Precip_17','APR_Depart_17','MAY_Precip_17','MAY_Depart_17','JUN_Precip_17','JUN_Depart_17','Station1','JUL_Precip_17','JUL_Depart_17','AUG_Precip_17','AUG_Depart_17','SEP_Precip_17','SEP_Depart_17','OCT_Precip_17','OCT_Depart_17','NOV_Precip_17','NOV_Depart_17','DEC_Precip_17','DEC_Depart_17']

MDrain2017 = MDrain2017.set_index('Station')
MDrain2017 = MDrain2017.drop(columns=['Station1'])

for x in MDrain2017.columns:  
    MDrain2017[x] = MDrain2017[x].str.replace(r'[A-Z]','')
    MDrain2017[x] = MDrain2017[x].str.replace('      ','0')
    MDrain2017[x] = MDrain2017[x].str.replace(' ','')
    MDrain2017[x] = MDrain2017[x].replace(r'', np.NaN)
    MDrain2017[x] = MDrain2017[x].astype('float')
MDrain2017 = MDrain2017.fillna(MDrain2017.mean())

print("MD Rain Data for 2017 cleaned and dataframe created")
print("")

#MD2016
file = path_a + 'MD2016.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2016 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2016.columns = ['Station','JAN_Precip_16','JAN_Depart_16','FEB_Precip_16','FEB_Depart_16','MAR_Precip_16','MAR_Depart_16','APR_Precip_16','APR_Depart_16','MAY_Precip_16','MAY_Depart_16','JUN_Precip_16','JUN_Depart_16','Station1','JUL_Precip_16','JUL_Depart_16','AUG_Precip_16','AUG_Depart_16','SEP_Precip_16','SEP_Depart_16','OCT_Precip_16','OCT_Depart_16','NOV_Precip_16','NOV_Depart_16','DEC_Precip_16','DEC_Depart_16']

MDrain2016 = MDrain2016.set_index('Station')
MDrain2016 = MDrain2016.drop(columns=['Station1'])

for x in MDrain2016.columns:  
    MDrain2016[x] = MDrain2016[x].str.replace(r'[A-Z]','')
    MDrain2016[x] = MDrain2016[x].str.replace('      ','0')
    MDrain2016[x] = MDrain2016[x].str.replace(' ','')
    MDrain2016[x] = MDrain2016[x].replace(r'', np.NaN)
    MDrain2016[x] = MDrain2016[x].astype('float')
MDrain2016 = MDrain2016.fillna(MDrain2016.mean())

print("MD Rain Data for 2016 cleaned and dataframe created")
print("")

#MD2015
file = path_a + 'MD2015.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2015 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2015.columns = ['Station','JAN_Precip_15','JAN_Depart_15','FEB_Precip_15','FEB_Depart_15','MAR_Precip_15','MAR_Depart_15','APR_Precip_15','APR_Depart_15','MAY_Precip_15','MAY_Depart_15','JUN_Precip_15','JUN_Depart_15','Station1','JUL_Precip_15','JUL_Depart_15','AUG_Precip_15','AUG_Depart_15','SEP_Precip_15','SEP_Depart_15','OCT_Precip_15','OCT_Depart_15','NOV_Precip_15','NOV_Depart_15','DEC_Precip_15','DEC_Depart_15']

MDrain2015 = MDrain2015.set_index('Station')
MDrain2015 = MDrain2015.drop(columns=['Station1'])

for x in MDrain2015.columns:  
    MDrain2015[x] = MDrain2015[x].str.replace(r'[A-Z]','')
    MDrain2015[x] = MDrain2015[x].str.replace('      ','0')
    MDrain2015[x] = MDrain2015[x].str.replace(' ','')
    MDrain2015[x] = MDrain2015[x].replace(r'', np.NaN)
    MDrain2015[x] = MDrain2015[x].astype('float')
MDrain2015 = MDrain2015.fillna(MDrain2015.mean())

print("MD Rain Data for 2015 cleaned and dataframe created")
print("")

#MD2014
file = path_a + 'MD2014.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2014 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2014.columns = ['Station','JAN_Precip_14','JAN_Depart_14','FEB_Precip_14','FEB_Depart_14','MAR_Precip_14','MAR_Depart_14','APR_Precip_14','APR_Depart_14','MAY_Precip_14','MAY_Depart_14','JUN_Precip_14','JUN_Depart_14','Station1','JUL_Precip_14','JUL_Depart_14','AUG_Precip_14','AUG_Depart_14','SEP_Precip_14','SEP_Depart_14','OCT_Precip_14','OCT_Depart_14','NOV_Precip_14','NOV_Depart_14','DEC_Precip_14','DEC_Depart_14']

MDrain2014 = MDrain2014.set_index('Station')
MDrain2014 = MDrain2014.drop(columns=['Station1'])

for x in MDrain2014.columns:  
    MDrain2014[x] = MDrain2014[x].str.replace(r'[A-Z]','')
    MDrain2014[x] = MDrain2014[x].str.replace('      ','0')
    MDrain2014[x] = MDrain2014[x].str.replace(' ','')
    MDrain2014[x] = MDrain2014[x].replace(r'', np.NaN)
    MDrain2014[x] = MDrain2014[x].astype('float')
MDrain2014 = MDrain2014.fillna(MDrain2014.mean())

print("MD Rain Data for 2014 cleaned and dataframe created")
print("")

#MD2013
file = path_a + 'MD2013.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2013 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2013.columns = ['Station','JAN_Precip_13','JAN_Depart_13','FEB_Precip_13','FEB_Depart_13','MAR_Precip_13','MAR_Depart_13','APR_Precip_13','APR_Depart_13','MAY_Precip_13','MAY_Depart_13','JUN_Precip_13','JUN_Depart_13','Station1','JUL_Precip_13','JUL_Depart_13','AUG_Precip_13','AUG_Depart_13','SEP_Precip_13','SEP_Depart_13','OCT_Precip_13','OCT_Depart_13','NOV_Precip_13','NOV_Depart_13','DEC_Precip_13','DEC_Depart_13']

MDrain2013 = MDrain2013.set_index('Station')
MDrain2013 = MDrain2013.drop(columns=['Station1'])

for x in MDrain2013.columns:  
    MDrain2013[x] = MDrain2013[x].str.replace(r'[A-Z]','')
    MDrain2013[x] = MDrain2013[x].str.replace('      ','0')
    MDrain2013[x] = MDrain2013[x].str.replace(' ','')
    MDrain2013[x] = MDrain2013[x].replace(r'', np.NaN)
    MDrain2013[x] = MDrain2013[x].astype('float')
MDrain2013 = MDrain2013.fillna(MDrain2013.mean())

print("MD Rain Data for 2013 cleaned and dataframe created")
print("")

#MD2012
file = path_a + 'MD2012.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2012 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2012.columns = ['Station','JAN_Precip_12','JAN_Depart_12','FEB_Precip_12','FEB_Depart_12','MAR_Precip_12','MAR_Depart_12','APR_Precip_12','APR_Depart_12','MAY_Precip_12','MAY_Depart_12','JUN_Precip_12','JUN_Depart_12','Station1','JUL_Precip_12','JUL_Depart_12','AUG_Precip_12','AUG_Depart_12','SEP_Precip_12','SEP_Depart_12','OCT_Precip_12','OCT_Depart_12','NOV_Precip_12','NOV_Depart_12','DEC_Precip_12','DEC_Depart_12']

MDrain2012 = MDrain2012.set_index('Station')
MDrain2012 = MDrain2012.drop(columns=['Station1'])

for x in MDrain2012.columns:  
    MDrain2012[x] = MDrain2012[x].str.replace(r'[A-Z]','')
    MDrain2012[x] = MDrain2012[x].str.replace('      ','0')
    MDrain2012[x] = MDrain2012[x].str.replace(' ','')
    MDrain2012[x] = MDrain2012[x].replace(r'', np.NaN)
    MDrain2012[x] = MDrain2012[x].astype('float')
MDrain2012 = MDrain2012.fillna(MDrain2012.mean())

print("MD Rain Data for 2012 cleaned and dataframe created")
print("")

#MD2011
file = path_a + 'MD2011.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2011 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2011.columns = ['Station','JAN_Precip_11','JAN_Depart_11','FEB_Precip_11','FEB_Depart_11','MAR_Precip_11','MAR_Depart_11','APR_Precip_11','APR_Depart_11','MAY_Precip_11','MAY_Depart_11','JUN_Precip_11','JUN_Depart_11','Station1','JUL_Precip_11','JUL_Depart_11','AUG_Precip_11','AUG_Depart_11','SEP_Precip_11','SEP_Depart_11','OCT_Precip_11','OCT_Depart_11','NOV_Precip_11','NOV_Depart_11','DEC_Precip_11','DEC_Depart_11']

MDrain2011 = MDrain2011.set_index('Station')
MDrain2011 = MDrain2011.drop(columns=['Station1'])

for x in MDrain2011.columns:  
    MDrain2011[x] = MDrain2011[x].str.replace(r'[A-Z]','')
    MDrain2011[x] = MDrain2011[x].str.replace('      ','0')
    MDrain2011[x] = MDrain2011[x].str.replace(' ','')
    MDrain2011[x] = MDrain2011[x].replace(r'', np.NaN)
    MDrain2011[x] = MDrain2011[x].astype('float')
MDrain2011 = MDrain2011.fillna(MDrain2011.mean())

print("MD Rain Data for 2011 cleaned and dataframe created")
print("")

#MD2010
file = path_a + 'MD2010.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2010 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2010.columns = ['Station','JAN_Precip_10','JAN_Depart_10','FEB_Precip_10','FEB_Depart_10','MAR_Precip_10','MAR_Depart_10','APR_Precip_10','APR_Depart_10','MAY_Precip_10','MAY_Depart_10','JUN_Precip_10','JUN_Depart_10','Station1','JUL_Precip_10','JUL_Depart_10','AUG_Precip_10','AUG_Depart_10','SEP_Precip_10','SEP_Depart_10','OCT_Precip_10','OCT_Depart_10','NOV_Precip_10','NOV_Depart_10','DEC_Precip_10','DEC_Depart_10']

MDrain2010 = MDrain2010.set_index('Station')
MDrain2010 = MDrain2010.drop(columns=['Station1'])

for x in MDrain2010.columns:  
    MDrain2010[x] = MDrain2010[x].str.replace(r'[A-Z]','')
    MDrain2010[x] = MDrain2010[x].str.replace('      ','0')
    MDrain2010[x] = MDrain2010[x].str.replace(' ','')
    MDrain2010[x] = MDrain2010[x].replace(r'', np.NaN)
    MDrain2010[x] = MDrain2010[x].astype('float')
MDrain2010 = MDrain2010.fillna(MDrain2010.mean())

print("MD Rain Data for 2010 cleaned and dataframe created")
print("")

#MD2009
file = path_a + 'MD2009.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2009 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2009.columns = ['Station','JAN_Precip_09','JAN_Depart_09','FEB_Precip_09','FEB_Depart_09','MAR_Precip_09','MAR_Depart_09','APR_Precip_09','APR_Depart_09','MAY_Precip_09','MAY_Depart_09','JUN_Precip_09','JUN_Depart_09','Station1','JUL_Precip_09','JUL_Depart_09','AUG_Precip_09','AUG_Depart_09','SEP_Precip_09','SEP_Depart_09','OCT_Precip_09','OCT_Depart_09','NOV_Precip_09','NOV_Depart_09','DEC_Precip_09','DEC_Depart_09']

MDrain2009 = MDrain2009.set_index('Station')
MDrain2009 = MDrain2009.drop(columns=['Station1'])

for x in MDrain2009.columns:  
    MDrain2009[x] = MDrain2009[x].str.replace(r'[A-Z]','')
    MDrain2009[x] = MDrain2009[x].str.replace('      ','0')
    MDrain2009[x] = MDrain2009[x].str.replace(' ','')
    MDrain2009[x] = MDrain2009[x].replace(r'', np.NaN)
    MDrain2009[x] = MDrain2009[x].astype('float')
MDrain2009 = MDrain2009.fillna(MDrain2009.mean())

print("MD Rain Data for 2009 cleaned and dataframe created")
print("")

#MD2008
file = path_a + 'MD2008.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables
try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2008 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2008.columns = ['Station','JAN_Precip_08','JAN_Depart_08','FEB_Precip_08','FEB_Depart_08','MAR_Precip_08','MAR_Depart_08','APR_Precip_08','APR_Depart_08','MAY_Precip_08','MAY_Depart_08','JUN_Precip_08','JUN_Depart_08','Station1','JUL_Precip_08','JUL_Depart_08','AUG_Precip_08','AUG_Depart_08','SEP_Precip_08','SEP_Depart_08','OCT_Precip_08','OCT_Depart_08','NOV_Precip_08','NOV_Depart_08','DEC_Precip_08','DEC_Depart_08']

MDrain2008 = MDrain2008.set_index('Station')
MDrain2008 = MDrain2008.drop(columns=['Station1'])

for x in MDrain2008.columns:  
    MDrain2008[x] = MDrain2008[x].str.replace(r'[A-Z]','')
    MDrain2008[x] = MDrain2008[x].str.replace('      ','0')
    MDrain2008[x] = MDrain2008[x].str.replace(' ','')
    MDrain2008[x] = MDrain2008[x].replace(r'', np.NaN)
    MDrain2008[x] = MDrain2008[x].astype('float')
MDrain2008 = MDrain2008.fillna(MDrain2008.mean())

print("MD Rain Data for 2008 cleaned and dataframe created")
print("")

#MD2007
file = path_a + 'MD2007.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2007 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2007.columns = ['Station','JAN_Precip_07','JAN_Depart_07','FEB_Precip_07','FEB_Depart_07','MAR_Precip_07','MAR_Depart_07','APR_Precip_07','APR_Depart_07','MAY_Precip_07','MAY_Depart_07','JUN_Precip_07','JUN_Depart_07','Station1','JUL_Precip_07','JUL_Depart_07','AUG_Precip_07','AUG_Depart_07','SEP_Precip_07','SEP_Depart_07','OCT_Precip_07','OCT_Depart_07','NOV_Precip_07','NOV_Depart_07','DEC_Precip_07','DEC_Depart_07']

MDrain2007 = MDrain2007.set_index('Station')
MDrain2007 = MDrain2007.drop(columns=['Station1'])

for x in MDrain2007.columns:  
    MDrain2007[x] = MDrain2007[x].str.replace(r'[A-Z]','')
    MDrain2007[x] = MDrain2007[x].str.replace('      ','0')
    MDrain2007[x] = MDrain2007[x].str.replace(' ','')
    MDrain2007[x] = MDrain2007[x].replace(r'', np.NaN)
    MDrain2007[x] = MDrain2007[x].astype('float')
MDrain2007 = MDrain2007.fillna(MDrain2007.mean())

print("MD Rain Data for 2007 cleaned and dataframe created")
print("")

#MD2006
file = path_a + 'MD2006.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2006 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2006.columns = ['Station','JAN_Precip_06','JAN_Depart_06','FEB_Precip_06','FEB_Depart_06','MAR_Precip_06','MAR_Depart_06','APR_Precip_06','APR_Depart_06','MAY_Precip_06','MAY_Depart_06','JUN_Precip_06','JUN_Depart_06','Station1','JUL_Precip_06','JUL_Depart_06','AUG_Precip_06','AUG_Depart_06','SEP_Precip_06','SEP_Depart_06','OCT_Precip_06','OCT_Depart_06','NOV_Precip_06','NOV_Depart_06','DEC_Precip_06','DEC_Depart_06']

MDrain2006 = MDrain2006.set_index('Station')
MDrain2006 = MDrain2006.drop(columns=['Station1'])

for x in MDrain2006.columns: 
    MDrain2006[x] = MDrain2006[x].str.replace(r'[A-Z]','')
    MDrain2006[x] = MDrain2006[x].str.replace('    ','0')
    MDrain2006[x] = MDrain2006[x].str.replace(' ','')
    MDrain2006[x] = MDrain2006[x].replace(r'', np.NaN)
    MDrain2006[x] = MDrain2006[x].astype('float')
MDrain2006 = MDrain2006.fillna(MDrain2006.mean())

print("MD Rain Data for 2006 cleaned and dataframe created")
print("")

#MD2005
file = path_a + 'MD2005.pdf'
tables = camelot.read_pdf(file, pages = '2,3')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jan1','Jan2','Feb1','Feb2','Mar1','Mar2','Apr1','Apr2','May1','May2','Jun1','Jun2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb1 = pd.DataFrame()
for i in range(2,15):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()
try1.columns = ['Station','Jul1','Jul2','Aug1','Aug2','Sep1','Sep2','Oct1','Oct2','Nov1','Nov2','Dec1','Dec2','Ann1','Ann2']

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1

comb2 = pd.DataFrame()
for i in range(2,15):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
MDrain2005 = pd.concat([comb1,comb2], axis=1, ignore_index = True,join='inner')
MDrain2005.columns = ['Station','JAN_Precip_05','JAN_Depart_05','FEB_Precip_05','FEB_Depart_05','MAR_Precip_05','MAR_Depart_05','APR_Precip_05','APR_Depart_05','MAY_Precip_05','MAY_Depart_05','JUN_Precip_05','JUN_Depart_05','Station1','JUL_Precip_05','JUL_Depart_05','AUG_Precip_05','AUG_Depart_05','SEP_Precip_05','SEP_Depart_05','OCT_Precip_05','OCT_Depart_05','NOV_Precip_05','NOV_Depart_05','DEC_Precip_05','DEC_Depart_05']

MDrain2005 = MDrain2005.set_index('Station')
MDrain2005 = MDrain2005.drop(columns=['Station1'])

for x in MDrain2005.columns:  
    MDrain2005[x] = MDrain2005[x].str.replace(r'[A-Z]','')
    MDrain2005[x] = MDrain2005[x].str.replace('      ','0')
    MDrain2005[x] = MDrain2005[x].str.replace(' ','')
    MDrain2005[x] = MDrain2005[x].astype('float')
MDrain2005 = MDrain2005.fillna(MDrain2005.mean())

print("MD Rain Data for 2005 cleaned and dataframe created")
print("")

MD Rain Data for 2017 cleaned and dataframe created

MD Rain Data for 2016 cleaned and dataframe created

MD Rain Data for 2015 cleaned and dataframe created

MD Rain Data for 2014 cleaned and dataframe created

MD Rain Data for 2013 cleaned and dataframe created

MD Rain Data for 2012 cleaned and dataframe created

MD Rain Data for 2011 cleaned and dataframe created

MD Rain Data for 2010 cleaned and dataframe created

MD Rain Data for 2009 cleaned and dataframe created

MD Rain Data for 2008 cleaned and dataframe created

MD Rain Data for 2007 cleaned and dataframe created

MD Rain Data for 2006 cleaned and dataframe created

MD Rain Data for 2005 cleaned and dataframe created



#### Combining all dataframes from 2005 to 2018 into one

In [18]:
Station = MDrain2005.index
MDrain = pd.DataFrame()
for i in range(2005,2019):
    MDrain.reset_index(drop=True, inplace=True)
    globals()['MDrain'+str(i)].reset_index(drop=True, inplace=True)
    MDrain = pd.concat([MDrain, globals()['MDrain'+str(i)]], axis =1)
    
MDrain = MDrain.set_index(Station)
MDrain = MDrain.transpose()
MDrain.to_csv(path_a+'\\MDrain.csv')

print("All dataframe combined and rain data for MD saved as .csv")

All dataframe combined and rain data for MD saved as .csv


In [19]:
MDrain

Station,SOUTHERN EASTERN,SHORE 01,ASSATEAGUE,PRINCESS ANNE,SALISBURY,SALISBURY POLICE BRKS,SALISBURY FAA AP,SNOW HILL 4 N,--DIVISIONAL DATA------->,CENTRAL EASTERN,...,ABERDEEN PHILLIPS FLD,CATOCTIN MOUNTAIN PARK,CONOWINGO DAM,CYLBURN,DAMASCUS 3 SSW,EMMITSBURG 2 SE,FREDERICK 2 NNE,MILLERS 4 NE,PORT DEPOSIT 2 NE,POTOMAC FILTER PLANT
JAN_Precip_05,4.510000,3.430000,4.88,5.04,4.04,4.19,4.370000,4.12,2.180000,4.12,...,1.520000,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556
JAN_Depart_05,0.860000,0.000000,-0.05,0.17,0.37,0.01,0.000000,0.14,-0.550000,0.00,...,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250
FEB_Precip_05,3.090000,2.940000,2.52,2.61,3.04,2.73,2.820000,2.25,2.100000,2.18,...,1.610000,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333
FEB_Depart_05,-0.500000,0.000000,-0.93,0.00,-0.52,-0.68,-0.600000,-1.10,-1.230000,-1.13,...,-0.970000,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167
MAR_Precip_05,4.600000,3.910000,4.69,2.85,4.05,4.84,4.420000,4.43,4.440000,4.44,...,5.000000,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OCT_Depart_18,3.500000,0.635385,5.18,-2.04,4.61,2.19,2.670000,4.57,0.635385,4.79,...,0.635385,0.635385,0.635385,0.635385,0.635385,0.635385,0.635385,0.635385,NaN,NaN
NOV_Precip_18,5.860000,6.390000,4.50,4.50,5.13,7.82,7.203226,6.89,8.330000,7.61,...,7.203226,7.203226,7.203226,7.203226,7.203226,7.203226,7.203226,7.203226,NaN,NaN
NOV_Depart_18,2.470000,3.734000,1.07,0.98,1.95,4.17,3.734000,3.30,3.734000,4.34,...,3.734000,3.734000,3.734000,3.734000,3.734000,3.734000,3.734000,3.734000,NaN,NaN
DEC_Precip_18,5.330345,4.670000,3.53,3.29,3.83,4.61,4.610000,4.92,5.330345,4.92,...,5.330345,5.330345,5.330345,5.330345,5.330345,5.330345,5.330345,5.330345,NaN,NaN


### Maryland Temperature Data

#### Essentially, we used the same technique for scraping and cleaning the temperature data as we did for precipitation data with some changes in cleaning techniques.

#### Maryland 2018

In [20]:
file = path_a + 'MD2018.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

<TableList n=2>

In [21]:
try1 = tables[0].df
try1

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,STATION,JAN,,FEB,,MAR,,APR,,MAY,...,SEP,,OCT,,NOV,,DEC,,ANNUAL,
1,,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.,TEMPERATURE,...,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.,TEMPERATURE,DEPARTURE.
2,MARYLAND\nSOUTHERN EASTERN SHORE 01\nPRINCESS...,M\n36.0\n33.8\n34.0\n34.6\n34.2\n34.2\n30.9M\n...,-1.6\n-3.3\n-1.4\n-2.1\n-0.6\n-3.1\n \n-3.1\n-...,M\n47.5\n45.6\n44.1M\n45.7\n43.9\n43.9\n42.2\n...,7.9\n4.6\n7.9\n5.0\n7.0\n5.3\n \n5.3\n6.4\n6.0...,M\nM\n41.1\n39.8\n40.5\n41.8\n41.8\n40.0\nMM\n...,-3.6\n-6.6\n-4.8\n-4.7\n-3.5\n-4.6\n \n-5.2\n-...,M\n54.1\n52.1\n51.5\n52.6\n53.3\n53.3\n51.5\nM...,-1.9\n-4.3\n-1.5\n-3.0\n-1.2\n-3.1\n \n-2.7\n-...,M\n71.4\n69.0\nM\n70.2\n70.9\n70.9\n68.6M\nMM\...,...,M\n76.5\n75.4\n78.3\n76.7\n75.3\n75.3\n72.9M\n...,7.2\n8.5\n7.4\n5.1\n6.1\n6.4\n \n4.8\n8.7\n5.2...,M\n62.2\n60.4\n63.8\n62.1\n62.3\n62.3\n57.8\nM...,3.2\n4.7\n3.8\n2.7\n4.1\n2.2\n \n0.9\n5.6\n3.2...,M\n47.5\n46.8\n50.9\n48.4\n46.6\n46.6\n44.2\nM...,-1.4\n0.9\n-0.4\n-3.4\n-1.5\n-2.9\n \n-3.1\n-1...,M\n43.1\n42.0\n45.5\n43.5\n41.9\n41.9\n39.8\nM...,3.0\n3.2\n1.8\n2.6\n \n1.0\n3.0\n2.3\n2.4\n3.6...,M \n61.0\n58.0\n4.8 M 58.5\n58.9\n58.6\n58.6\n...,2.0\n0.9\n2.5\n0.7\n2.2\n0.9\n0.3\n4.2\n1.6\n0...


In [22]:
#Cleaning the first table on sixth page
try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_18','Jan_Depart_18','Feb_Temp_18','Feb_Depart_18','Mar_Temp_18','Mar_Depart_18','Apr_Temp_18','Apr_Depart_18','May_Temp_18','May_Depart_18','Jun_Temp_18','Jun_Depart_18','Jul_Temp_18','Jul_Depart_18','Aug_Temp_18','Aug_Depart_18','Sep_Temp_18','Sep_Depart_18','Oct_Temp_18','Oct_Depart_18','Nov_Temp_18','Nov_Depart_18','Dec_Temp_18','Dec_Depart_18','Ann_Temp_18','Ann_Depart_18']

In [23]:
#Cleaning the second table on seventh page
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_18','Jan_Depart_18','Feb_Temp_18','Feb_Depart_18','Mar_Temp_18','Mar_Depart_18','Apr_Temp_18','Apr_Depart_18','May_Temp_18','May_Depart_18','Jun_Temp_18','Jun_Depart_18','Jul_Temp_18','Jul_Depart_18','Aug_Temp_18','Aug_Depart_18','Sep_Temp_18','Sep_Depart_18','Oct_Temp_18','Oct_Depart_18','Nov_Temp_18','Nov_Depart_18','Dec_Temp_18','Dec_Depart_18','Ann_Temp_18','Ann_Depart_18']

In [24]:
MDtemp2018 = pd.concat([comb1,comb2])
MDtemp2018 = MDtemp2018.set_index('Station')

for x in MDtemp2018.columns: 
    MDtemp2018[x] = MDtemp2018[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2018.columns:  
    MDtemp2018[x] = MDtemp2018[x].str.replace(r'[A-Z]','')
    MDtemp2018[x] = MDtemp2018[x].str.replace('      ','0')
    MDtemp2018[x] = MDtemp2018[x].str.replace(' ','')
    MDtemp2018[x] = MDtemp2018[x].replace(r'', np.NaN)
    MDtemp2018[x] = MDtemp2018[x].astype('float')
MDtemp2018 = MDtemp2018.fillna(MDtemp2018.mean())

print("MD Temperature Data for 2018 cleaned and dataframe created")
print("")

MD Temperature Data for 2018 cleaned and dataframe created



In [25]:
MDtemp2018.head()

,Jan_Temp_18,Jan_Depart_18,Feb_Temp_18,Feb_Depart_18,Mar_Temp_18,Mar_Depart_18,Apr_Temp_18,Apr_Depart_18,May_Temp_18,May_Depart_18,...,Sep_Temp_18,Sep_Depart_18,Oct_Temp_18,Oct_Depart_18,Nov_Temp_18,Nov_Depart_18,Dec_Temp_18,Dec_Depart_18,Ann_Temp_18,Ann_Depart_18
Station,,,,,,,,,,,,,,,,,,,,,
MARYLAND,30.342105,-1.0,40.589744,7.0,38.205128,-3.0,50.128205,-1.0,68.0,6.000000,...,72.358974,7.0,58.45,3.0,42.85,-1.0,38.615385,3.000000,48.138889,2.0
SOUTHERN EASTERN SHORE 01,36.000000,-3.0,47.000000,4.0,38.205128,-6.0,54.000000,-4.0,71.0,5.628571,...,76.000000,8.0,62.00,4.0,47.00,0.0,43.000000,3.000000,61.000000,0.0
PRINCESS ANNE,33.000000,-1.0,45.000000,7.0,41.000000,-4.0,52.000000,-1.0,69.0,7.000000,...,75.000000,7.0,60.00,3.0,46.00,-0.0,42.000000,1.000000,58.000000,2.0
SALISBURY 2N,34.000000,-2.0,44.000000,5.0,39.000000,-4.0,51.000000,-3.0,68.0,5.000000,...,78.000000,5.0,63.00,2.0,50.00,-3.0,45.000000,2.000000,4.000000,0.0
SALISBURY-WICOMICO RGNL AP,34.000000,-0.0,45.000000,7.0,40.000000,-3.0,52.000000,-1.0,70.0,6.000000,...,76.000000,6.0,62.00,4.0,48.00,-1.0,43.000000,2.096774,58.000000,2.0


#### Same process is repeated for years 2005 - 2017

In [26]:
#MD2017
file = path_a + 'MD2017.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_17','Jan_Depart_17','Feb_Temp_17','Feb_Depart_17','Mar_Temp_17','Mar_Depart_17','Apr_Temp_17','Apr_Depart_17','May_Temp_17','May_Depart_17','Jun_Temp_17','Jun_Depart_17','Jul_Temp_17','Jul_Depart_17','Aug_Temp_17','Aug_Depart_17','Sep_Temp_17','Sep_Depart_17','Oct_Temp_17','Oct_Depart_17','Nov_Temp_17','Nov_Depart_17','Dec_Temp_17','Dec_Depart_17','Ann_Temp_17','Ann_Depart_17']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_17','Jan_Depart_17','Feb_Temp_17','Feb_Depart_17','Mar_Temp_17','Mar_Depart_17','Apr_Temp_17','Apr_Depart_17','May_Temp_17','May_Depart_17','Jun_Temp_17','Jun_Depart_17','Jul_Temp_17','Jul_Depart_17','Aug_Temp_17','Aug_Depart_17','Sep_Temp_17','Sep_Depart_17','Oct_Temp_17','Oct_Depart_17','Nov_Temp_17','Nov_Depart_17','Dec_Temp_17','Dec_Depart_17','Ann_Temp_17','Ann_Depart_17']

MDtemp2017 = pd.concat([comb1,comb2])
MDtemp2017 = MDtemp2017.set_index('Station')

for x in MDtemp2017.columns: 
    MDtemp2017[x] = MDtemp2017[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2017.columns:  
    MDtemp2017[x] = MDtemp2017[x].str.replace(r'[A-Z]','')
    MDtemp2017[x] = MDtemp2017[x].str.replace('      ','0')
    MDtemp2017[x] = MDtemp2017[x].str.replace(' ','')
    MDtemp2017[x] = MDtemp2017[x].replace(r'', np.NaN)
    MDtemp2017[x] = MDtemp2017[x].astype('float')
MDtemp2017 = MDtemp2017.fillna(MDtemp2017.mean())

print("MD Temperature Data for 2017 cleaned and dataframe created")
print("")

#MD2016
file = path_a + 'MD2016.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_16','Jan_Depart_16','Feb_Temp_16','Feb_Depart_16','Mar_Temp_16','Mar_Depart_16','Apr_Temp_16','Apr_Depart_16','May_Temp_16','May_Depart_16','Jun_Temp_16','Jun_Depart_16','Jul_Temp_16','Jul_Depart_16','Aug_Temp_16','Aug_Depart_16','Sep_Temp_16','Sep_Depart_16','Oct_Temp_16','Oct_Depart_16','Nov_Temp_16','Nov_Depart_16','Dec_Temp_16','Dec_Depart_16','Ann_Temp_16','Ann_Depart_16']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_16','Jan_Depart_16','Feb_Temp_16','Feb_Depart_16','Mar_Temp_16','Mar_Depart_16','Apr_Temp_16','Apr_Depart_16','May_Temp_16','May_Depart_16','Jun_Temp_16','Jun_Depart_16','Jul_Temp_16','Jul_Depart_16','Aug_Temp_16','Aug_Depart_16','Sep_Temp_16','Sep_Depart_16','Oct_Temp_16','Oct_Depart_16','Nov_Temp_16','Nov_Depart_16','Dec_Temp_16','Dec_Depart_16','Ann_Temp_16','Ann_Depart_16']

MDtemp2016 = pd.concat([comb1,comb2])
MDtemp2016 = MDtemp2016.set_index('Station')

for x in MDtemp2016.columns: 
    MDtemp2016[x] = MDtemp2016[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2016.columns:  
    MDtemp2016[x] = MDtemp2016[x].str.replace(r'[A-Z]','')
    MDtemp2016[x] = MDtemp2016[x].str.replace('      ','0')
    MDtemp2016[x] = MDtemp2016[x].str.replace(' ','')
    MDtemp2016[x] = MDtemp2016[x].replace(r'', np.NaN)
    MDtemp2016[x] = MDtemp2016[x].astype('float')
MDtemp2016 = MDtemp2016.fillna(MDtemp2016.mean())
print("MD Temperature Data for 2016 cleaned and dataframe created")
print("")

#MD2015
file = path_a + 'MD2015.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_15','Jan_Depart_15','Feb_Temp_15','Feb_Depart_15','Mar_Temp_15','Mar_Depart_15','Apr_Temp_15','Apr_Depart_15','May_Temp_15','May_Depart_15','Jun_Temp_15','Jun_Depart_15','Jul_Temp_15','Jul_Depart_15','Aug_Temp_15','Aug_Depart_15','Sep_Temp_15','Sep_Depart_15','Oct_Temp_15','Oct_Depart_15','Nov_Temp_15','Nov_Depart_15','Dec_Temp_15','Dec_Depart_15','Ann_Temp_15','Ann_Depart_15']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_15','Jan_Depart_15','Feb_Temp_15','Feb_Depart_15','Mar_Temp_15','Mar_Depart_15','Apr_Temp_15','Apr_Depart_15','May_Temp_15','May_Depart_15','Jun_Temp_15','Jun_Depart_15','Jul_Temp_15','Jul_Depart_15','Aug_Temp_15','Aug_Depart_15','Sep_Temp_15','Sep_Depart_15','Oct_Temp_15','Oct_Depart_15','Nov_Temp_15','Nov_Depart_15','Dec_Temp_15','Dec_Depart_15','Ann_Temp_15','Ann_Depart_15']

MDtemp2015 = pd.concat([comb1,comb2])
MDtemp2015 = MDtemp2015.set_index('Station')

for x in MDtemp2015.columns: 
    MDtemp2015[x] = MDtemp2015[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2015.columns:  
    MDtemp2015[x] = MDtemp2015[x].str.replace(r'[A-Z]','')
    MDtemp2015[x] = MDtemp2015[x].str.replace('      ','0')
    MDtemp2015[x] = MDtemp2015[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2015[x] = MDtemp2015[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2015[x] = MDtemp2015[x].str.replace(' ','')
    MDtemp2015[x] = MDtemp2015[x].replace(r'', np.NaN)
    MDtemp2015[x] = MDtemp2015[x].astype('float')
MDtemp2015 = MDtemp2015.fillna(MDtemp2015.mean())

print("MD Temperature Data for 2015 cleaned and dataframe created")
print("")
 
#MD2014
file = path_a + 'MD2014.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_14','Jan_Depart_14','Feb_Temp_14','Feb_Depart_14','Mar_Temp_14','Mar_Depart_14','Apr_Temp_14','Apr_Depart_14','May_Temp_14','May_Depart_14','Jun_Temp_14','Jun_Depart_14','Jul_Temp_14','Jul_Depart_14','Aug_Temp_14','Aug_Depart_14','Sep_Temp_14','Sep_Depart_14','Oct_Temp_14','Oct_Depart_14','Nov_Temp_14','Nov_Depart_14','Dec_Temp_14','Dec_Depart_14','Ann_Temp_14','Ann_Depart_14']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_14','Jan_Depart_14','Feb_Temp_14','Feb_Depart_14','Mar_Temp_14','Mar_Depart_14','Apr_Temp_14','Apr_Depart_14','May_Temp_14','May_Depart_14','Jun_Temp_14','Jun_Depart_14','Jul_Temp_14','Jul_Depart_14','Aug_Temp_14','Aug_Depart_14','Sep_Temp_14','Sep_Depart_14','Oct_Temp_14','Oct_Depart_14','Nov_Temp_14','Nov_Depart_14','Dec_Temp_14','Dec_Depart_14','Ann_Temp_14','Ann_Depart_14']

MDtemp2014 = pd.concat([comb1,comb2])
MDtemp2014 = MDtemp2014.set_index('Station')

for x in MDtemp2014.columns: 
    MDtemp2014[x] = MDtemp2014[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2014.columns:  
    MDtemp2014[x] = MDtemp2014[x].str.replace(r'[A-Z]','')
    MDtemp2014[x] = MDtemp2014[x].str.replace('      ','0')
    MDtemp2014[x] = MDtemp2014[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2014[x] = MDtemp2014[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2014[x] = MDtemp2014[x].str.replace(' ','')
    MDtemp2014[x] = MDtemp2014[x].replace(r'', np.NaN)
    MDtemp2014[x] = MDtemp2014[x].astype('float')
MDtemp2014 = MDtemp2014.fillna(MDtemp2014.mean())

print("MD Temperature Data for 2014 cleaned and dataframe created")
print("")

#MD2013
file = path_a + 'MD2013.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_13','Jan_Depart_13','Feb_Temp_13','Feb_Depart_13','Mar_Temp_13','Mar_Depart_13','Apr_Temp_13','Apr_Depart_13','May_Temp_13','May_Depart_13','Jun_Temp_13','Jun_Depart_13','Jul_Temp_13','Jul_Depart_13','Aug_Temp_13','Aug_Depart_13','Sep_Temp_13','Sep_Depart_13','Oct_Temp_13','Oct_Depart_13','Nov_Temp_13','Nov_Depart_13','Dec_Temp_13','Dec_Depart_13','Ann_Temp_13','Ann_Depart_13']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_13','Jan_Depart_13','Feb_Temp_13','Feb_Depart_13','Mar_Temp_13','Mar_Depart_13','Apr_Temp_13','Apr_Depart_13','May_Temp_13','May_Depart_13','Jun_Temp_13','Jun_Depart_13','Jul_Temp_13','Jul_Depart_13','Aug_Temp_13','Aug_Depart_13','Sep_Temp_13','Sep_Depart_13','Oct_Temp_13','Oct_Depart_13','Nov_Temp_13','Nov_Depart_13','Dec_Temp_13','Dec_Depart_13','Ann_Temp_13','Ann_Depart_13']

MDtemp2013 = pd.concat([comb1,comb2])
MDtemp2013 = MDtemp2013.set_index('Station')

for x in MDtemp2013.columns: 
    MDtemp2013[x] = MDtemp2013[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2013.columns:  
    MDtemp2013[x] = MDtemp2013[x].str.replace(r'[A-Z]','')
    MDtemp2013[x] = MDtemp2013[x].str.replace('      ','0')
    MDtemp2013[x] = MDtemp2013[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2013[x] = MDtemp2013[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2013[x] = MDtemp2013[x].str.replace(' ','')
    MDtemp2013[x] = MDtemp2013[x].replace(r'', np.NaN)
    MDtemp2013[x] = MDtemp2013[x].astype('float')
MDtemp2013 = MDtemp2013.fillna(MDtemp2013.mean())

print("MD Temperature Data for 2013 cleaned and dataframe created")
print("")

#MD2012
file = path_a + 'MD2012.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_12','Jan_Depart_12','Feb_Temp_12','Feb_Depart_12','Mar_Temp_12','Mar_Depart_12','Apr_Temp_12','Apr_Depart_12','May_Temp_12','May_Depart_12','Jun_Temp_12','Jun_Depart_12','Jul_Temp_12','Jul_Depart_12','Aug_Temp_12','Aug_Depart_12','Sep_Temp_12','Sep_Depart_12','Oct_Temp_12','Oct_Depart_12','Nov_Temp_12','Nov_Depart_12','Dec_Temp_12','Dec_Depart_12','Ann_Temp_12','Ann_Depart_12']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_12','Jan_Depart_12','Feb_Temp_12','Feb_Depart_12','Mar_Temp_12','Mar_Depart_12','Apr_Temp_12','Apr_Depart_12','May_Temp_12','May_Depart_12','Jun_Temp_12','Jun_Depart_12','Jul_Temp_12','Jul_Depart_12','Aug_Temp_12','Aug_Depart_12','Sep_Temp_12','Sep_Depart_12','Oct_Temp_12','Oct_Depart_12','Nov_Temp_12','Nov_Depart_12','Dec_Temp_12','Dec_Depart_12','Ann_Temp_12','Ann_Depart_12']

MDtemp2012 = pd.concat([comb1,comb2])
MDtemp2012 = MDtemp2012.set_index('Station')

for x in MDtemp2012.columns: 
    MDtemp2012[x] = MDtemp2012[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2012.columns:  
    MDtemp2012[x] = MDtemp2012[x].str.replace(r'[A-Z]','')
    MDtemp2012[x] = MDtemp2012[x].str.replace('      ','0')
    MDtemp2012[x] = MDtemp2012[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2012[x] = MDtemp2012[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2012[x] = MDtemp2012[x].str.replace(' ','')
    MDtemp2012[x] = MDtemp2012[x].replace(r'', np.NaN)
    MDtemp2012[x] = MDtemp2012[x].astype('float')
MDtemp2012 = MDtemp2012.fillna(MDtemp2012.mean())

print("MD Temperature Data for 2012 cleaned and dataframe created")
print("")

#MD2011
file = path_a + 'MD2011.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_11','Jan_Depart_11','Feb_Temp_11','Feb_Depart_11','Mar_Temp_11','Mar_Depart_11','Apr_Temp_11','Apr_Depart_11','May_Temp_11','May_Depart_11','Jun_Temp_11','Jun_Depart_11','Jul_Temp_11','Jul_Depart_11','Aug_Temp_11','Aug_Depart_11','Sep_Temp_11','Sep_Depart_11','Oct_Temp_11','Oct_Depart_11','Nov_Temp_11','Nov_Depart_11','Dec_Temp_11','Dec_Depart_11','Ann_Temp_11','Ann_Depart_11']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_11','Jan_Depart_11','Feb_Temp_11','Feb_Depart_11','Mar_Temp_11','Mar_Depart_11','Apr_Temp_11','Apr_Depart_11','May_Temp_11','May_Depart_11','Jun_Temp_11','Jun_Depart_11','Jul_Temp_11','Jul_Depart_11','Aug_Temp_11','Aug_Depart_11','Sep_Temp_11','Sep_Depart_11','Oct_Temp_11','Oct_Depart_11','Nov_Temp_11','Nov_Depart_11','Dec_Temp_11','Dec_Depart_11','Ann_Temp_11','Ann_Depart_11']

MDtemp2011 = pd.concat([comb1,comb2])
MDtemp2011 = MDtemp2011.set_index('Station')

for x in MDtemp2011.columns: 
    MDtemp2011[x] = MDtemp2011[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2011.columns:  
    MDtemp2011[x] = MDtemp2011[x].str.replace(r'[A-Z]','')
    MDtemp2011[x] = MDtemp2011[x].str.replace('      ','0')
    MDtemp2011[x] = MDtemp2011[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2011[x] = MDtemp2011[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2011[x] = MDtemp2011[x].str.replace(' ','')
    MDtemp2011[x] = MDtemp2011[x].replace(r'', np.NaN)
    MDtemp2011[x] = MDtemp2011[x].astype('float')
MDtemp2011 = MDtemp2011.fillna(MDtemp2011.mean())

print("MD Temperature Data for 2011 cleaned and dataframe created")
print("")
 
#MD2010
file = path_a + 'MD2010.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_10','Jan_Depart_10','Feb_Temp_10','Feb_Depart_10','Mar_Temp_10','Mar_Depart_10','Apr_Temp_10','Apr_Depart_10','May_Temp_10','May_Depart_10','Jun_Temp_10','Jun_Depart_10','Jul_Temp_10','Jul_Depart_10','Aug_Temp_10','Aug_Depart_10','Sep_Temp_10','Sep_Depart_10','Oct_Temp_10','Oct_Depart_10','Nov_Temp_10','Nov_Depart_10','Dec_Temp_10','Dec_Depart_10','Ann_Temp_10','Ann_Depart_10']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_10','Jan_Depart_10','Feb_Temp_10','Feb_Depart_10','Mar_Temp_10','Mar_Depart_10','Apr_Temp_10','Apr_Depart_10','May_Temp_10','May_Depart_10','Jun_Temp_10','Jun_Depart_10','Jul_Temp_10','Jul_Depart_10','Aug_Temp_10','Aug_Depart_10','Sep_Temp_10','Sep_Depart_10','Oct_Temp_10','Oct_Depart_10','Nov_Temp_10','Nov_Depart_10','Dec_Temp_10','Dec_Depart_10','Ann_Temp_10','Ann_Depart_10']

MDtemp2010 = pd.concat([comb1,comb2])
MDtemp2010 = MDtemp2010.set_index('Station')

for x in MDtemp2010.columns: 
    MDtemp2010[x] = MDtemp2010[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2010.columns:  
    MDtemp2010[x] = MDtemp2010[x].str.replace(r'[A-Z]','')
    MDtemp2010[x] = MDtemp2010[x].str.replace('      ','0')
    MDtemp2010[x] = MDtemp2010[x].str.replace('-','')
    MDtemp2010[x] = MDtemp2010[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2010[x] = MDtemp2010[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2010[x] = MDtemp2010[x].str.replace(' ','')
    MDtemp2010[x] = MDtemp2010[x].replace(r'', np.NaN)
    MDtemp2010[x] = MDtemp2010[x].astype('float')
MDtemp2010 = MDtemp2010.fillna(MDtemp2010.mean())

print("MD Temperature Data for 2010 cleaned and dataframe created")
print("")
 
#MD2009
file = path_a + 'MD2009.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_09','Jan_Depart_09','Feb_Temp_09','Feb_Depart_09','Mar_Temp_09','Mar_Depart_09','Apr_Temp_09','Apr_Depart_09','May_Temp_09','May_Depart_09','Jun_Temp_09','Jun_Depart_09','Jul_Temp_09','Jul_Depart_09','Aug_Temp_09','Aug_Depart_09','Sep_Temp_09','Sep_Depart_09','Oct_Temp_09','Oct_Depart_09','Nov_Temp_09','Nov_Depart_09','Dec_Temp_09','Dec_Depart_09','Ann_Temp_09','Ann_Depart_09']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_09','Jan_Depart_09','Feb_Temp_09','Feb_Depart_09','Mar_Temp_09','Mar_Depart_09','Apr_Temp_09','Apr_Depart_09','May_Temp_09','May_Depart_09','Jun_Temp_09','Jun_Depart_09','Jul_Temp_09','Jul_Depart_09','Aug_Temp_09','Aug_Depart_09','Sep_Temp_09','Sep_Depart_09','Oct_Temp_09','Oct_Depart_09','Nov_Temp_09','Nov_Depart_09','Dec_Temp_09','Dec_Depart_09','Ann_Temp_09','Ann_Depart_09']

MDtemp2009 = pd.concat([comb1,comb2])
MDtemp2009 = MDtemp2009.set_index('Station')

for x in MDtemp2009.columns: 
    MDtemp2009[x] = MDtemp2009[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2009.columns:  
    MDtemp2009[x] = MDtemp2009[x].str.replace(r'[A-Z]','')
    MDtemp2009[x] = MDtemp2009[x].str.replace('-','')
    MDtemp2009[x] = MDtemp2009[x].str.replace('      ','0')
    MDtemp2009[x] = MDtemp2009[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2009[x] = MDtemp2009[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2009[x] = MDtemp2009[x].str.replace(' ','')
    MDtemp2009[x] = MDtemp2009[x].replace(r'', np.NaN)
    MDtemp2009[x] = MDtemp2009[x].astype('float')
MDtemp2009 = MDtemp2009.fillna(MDtemp2009.mean())

print("MD Temperature Data for 2009 cleaned and dataframe created")
print("")
 
#MD2008  
file = path_a + 'MD2008.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_08','Jan_Depart_08','Feb_Temp_08','Feb_Depart_08','Mar_Temp_08','Mar_Depart_08','Apr_Temp_08','Apr_Depart_08','May_Temp_08','May_Depart_08','Jun_Temp_08','Jun_Depart_08','Jul_Temp_08','Jul_Depart_08','Aug_Temp_08','Aug_Depart_08','Sep_Temp_08','Sep_Depart_08','Oct_Temp_08','Oct_Depart_08','Nov_Temp_08','Nov_Depart_08','Dec_Temp_08','Dec_Depart_08','Ann_Temp_08','Ann_Depart_08']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_08','Jan_Depart_08','Feb_Temp_08','Feb_Depart_08','Mar_Temp_08','Mar_Depart_08','Apr_Temp_08','Apr_Depart_08','May_Temp_08','May_Depart_08','Jun_Temp_08','Jun_Depart_08','Jul_Temp_08','Jul_Depart_08','Aug_Temp_08','Aug_Depart_08','Sep_Temp_08','Sep_Depart_08','Oct_Temp_08','Oct_Depart_08','Nov_Temp_08','Nov_Depart_08','Dec_Temp_08','Dec_Depart_08','Ann_Temp_08','Ann_Depart_08']

MDtemp2008 = pd.concat([comb1,comb2])
MDtemp2008 = MDtemp2008.set_index('Station')

for x in MDtemp2008.columns: 
    MDtemp2008[x] = MDtemp2008[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2008.columns:  
    MDtemp2008[x] = MDtemp2008[x].str.replace(r'[A-Z]','')
    MDtemp2008[x] = MDtemp2008[x].str.replace('-','')
    MDtemp2008[x] = MDtemp2008[x].str.replace('      ','0')
    MDtemp2008[x] = MDtemp2008[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2008[x] = MDtemp2008[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2008[x] = MDtemp2008[x].str.replace(' ','')
    MDtemp2008[x] = MDtemp2008[x].replace(r'', np.NaN)
    MDtemp2008[x] = MDtemp2008[x].astype('float')
MDtemp2008 = MDtemp2008.fillna(MDtemp2008.mean())

print("MD Temperature Data for 2008 cleaned and dataframe created")
print("")
 
#MD2007  
file = path_a + 'MD2007.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_07','Jan_Depart_07','Feb_Temp_07','Feb_Depart_07','Mar_Temp_07','Mar_Depart_07','Apr_Temp_07','Apr_Depart_07','May_Temp_07','May_Depart_07','Jun_Temp_07','Jun_Depart_07','Jul_Temp_07','Jul_Depart_07','Aug_Temp_07','Aug_Depart_07','Sep_Temp_07','Sep_Depart_07','Oct_Temp_07','Oct_Depart_07','Nov_Temp_07','Nov_Depart_07','Dec_Temp_07','Dec_Depart_07','Ann_Temp_07','Ann_Depart_07']
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_07','Jan_Depart_07','Feb_Temp_07','Feb_Depart_07','Mar_Temp_07','Mar_Depart_07','Apr_Temp_07','Apr_Depart_07','May_Temp_07','May_Depart_07','Jun_Temp_07','Jun_Depart_07','Jul_Temp_07','Jul_Depart_07','Aug_Temp_07','Aug_Depart_07','Sep_Temp_07','Sep_Depart_07','Oct_Temp_07','Oct_Depart_07','Nov_Temp_07','Nov_Depart_07','Dec_Temp_07','Dec_Depart_07','Ann_Temp_07','Ann_Depart_07']

MDtemp2007 = pd.concat([comb1,comb2])
MDtemp2007 = MDtemp2007.set_index('Station')

for x in MDtemp2007.columns: 
    MDtemp2007[x] = MDtemp2007[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2007.columns:  
    MDtemp2007[x] = MDtemp2007[x].str.replace(r'[A-Z]','')
    MDtemp2007[x] = MDtemp2007[x].str.replace('-','')
    MDtemp2007[x] = MDtemp2007[x].str.replace('      ','0')
    MDtemp2007[x] = MDtemp2007[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2007[x] = MDtemp2007[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2007[x] = MDtemp2007[x].str.replace(' ','')
    MDtemp2007[x] = MDtemp2007[x].replace(r'', np.NaN)
    MDtemp2007[x] = MDtemp2007[x].astype('float')
MDtemp2007 = MDtemp2007.fillna(MDtemp2007.mean())

print("MD Temperature Data for 2007 cleaned and dataframe created")
print("")
 
#MD2006
file = path_a + 'MD2006.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_06','Jan_Depart_06','Feb_Temp_06','Feb_Depart_06','Mar_Temp_06','Mar_Depart_06','Apr_Temp_06','Apr_Depart_06','May_Temp_06','May_Depart_06','Jun_Temp_06','Jun_Depart_06','Jul_Temp_06','Jul_Depart_06','Aug_Temp_06','Aug_Depart_06','Sep_Temp_06','Sep_Depart_06','Oct_Temp_06','Oct_Depart_06','Nov_Temp_06','Nov_Depart_06','Dec_Temp_06','Dec_Depart_06','Ann_Temp_06','Ann_Depart_06']

try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_06','Jan_Depart_06','Feb_Temp_06','Feb_Depart_06','Mar_Temp_06','Mar_Depart_06','Apr_Temp_06','Apr_Depart_06','May_Temp_06','May_Depart_06','Jun_Temp_06','Jun_Depart_06','Jul_Temp_06','Jul_Depart_06','Aug_Temp_06','Aug_Depart_06','Sep_Temp_06','Sep_Depart_06','Oct_Temp_06','Oct_Depart_06','Nov_Temp_06','Nov_Depart_06','Dec_Temp_06','Dec_Depart_06','Ann_Temp_06','Ann_Depart_06']

MDtemp2006 = pd.concat([comb1,comb2])
MDtemp2006 = MDtemp2006.set_index('Station')

for x in MDtemp2006.columns: 
    MDtemp2006[x] = MDtemp2006[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2006.columns:  
    MDtemp2006[x] = MDtemp2006[x].str.replace(r'[A-Z]','')
    MDtemp2006[x] = MDtemp2006[x].str.replace('      ','0')
    MDtemp2006[x] = MDtemp2006[x].str.replace('-','')
    MDtemp2006[x] = MDtemp2006[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2006[x] = MDtemp2006[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2006[x] = MDtemp2006[x].str.replace(' ','')
    MDtemp2006[x] = MDtemp2006[x].replace(r'', np.NaN)
    MDtemp2006[x] = MDtemp2006[x].astype('float')
MDtemp2006 = MDtemp2006.fillna(MDtemp2006.mean())

print("MD Temperature Data for 2006 cleaned and dataframe created")
print("")
 
#MD2005    
file = path_a + 'MD2005.pdf'
tables = camelot.read_pdf(file, pages = '6,7')
tables

try1 = tables[0].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb1 = pd.DataFrame()
for i in range(2,29):
    comb1 = pd.concat([comb1, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb1.columns = ['Station','Jan_Temp_05','Jan_Depart_05','Feb_Temp_05','Feb_Depart_05','Mar_Temp_05','Mar_Depart_05','Apr_Temp_05','Apr_Depart_05','May_Temp_05','May_Depart_05','Jun_Temp_05','Jun_Depart_05','Jul_Temp_05','Jul_Depart_05','Aug_Temp_05','Aug_Depart_05','Sep_Temp_05','Sep_Depart_05','Oct_Temp_05','Oct_Depart_05','Nov_Temp_05','Nov_Depart_05','Dec_Temp_05','Dec_Depart_05','Ann_Temp_05','Ann_Depart_05']
try1 = tables[1].df
try1 = pd.DataFrame(try1.iloc[2])
try1 = try1.transpose()

i=2
for x in try1.columns:
    globals()['try'+str(i)] = pd.DataFrame(try1[x].str.split('\n').tolist()).stack()
    i=i+1
    
comb2 = pd.DataFrame()
for i in range(2,29):
    comb2 = pd.concat([comb2, globals()['try'+str(i)]], ignore_index=True, axis =1)
    
comb2.columns = ['Station','Jan_Temp_05','Jan_Depart_05','Feb_Temp_05','Feb_Depart_05','Mar_Temp_05','Mar_Depart_05','Apr_Temp_05','Apr_Depart_05','May_Temp_05','May_Depart_05','Jun_Temp_05','Jun_Depart_05','Jul_Temp_05','Jul_Depart_05','Aug_Temp_05','Aug_Depart_05','Sep_Temp_05','Sep_Depart_05','Oct_Temp_05','Oct_Depart_05','Nov_Temp_05','Nov_Depart_05','Dec_Temp_05','Dec_Depart_05','Ann_Temp_05','Ann_Depart_05']

MDtemp2005 = pd.concat([comb1,comb2])
MDtemp2005 = MDtemp2005.set_index('Station')

for x in MDtemp2005.columns: 
    MDtemp2005[x] = MDtemp2005[x].str.split(".", n = 1, expand = True) 
    
for x in MDtemp2005.columns:  
    MDtemp2005[x] = MDtemp2005[x].str.replace(r'[A-Z]','')
    MDtemp2005[x] = MDtemp2005[x].str.replace('-','')
    MDtemp2005[x] = MDtemp2005[x].str.replace('      ','0')
    MDtemp2005[x] = MDtemp2005[x].str.replace(r'(?<![0-9])\.\.(?<![0-9])','.')
    MDtemp2005[x] = MDtemp2005[x].str.replace(r'(?<![0-9])\.(?<![0-9])','')
    MDtemp2005[x] = MDtemp2005[x].str.replace(' ','')
    MDtemp2005[x] = MDtemp2005[x].replace(r'', np.NaN)
    MDtemp2005[x] = MDtemp2005[x].astype('float')
MDtemp2005 = MDtemp2005.fillna(MDtemp2005.mean())

print("MD Temperature Data for 2005 cleaned and dataframe created")
print("")

MD Temperature Data for 2017 cleaned and dataframe created

MD Temperature Data for 2016 cleaned and dataframe created

MD Temperature Data for 2015 cleaned and dataframe created

MD Temperature Data for 2014 cleaned and dataframe created

MD Temperature Data for 2013 cleaned and dataframe created

MD Temperature Data for 2012 cleaned and dataframe created

MD Temperature Data for 2011 cleaned and dataframe created

MD Temperature Data for 2010 cleaned and dataframe created

MD Temperature Data for 2009 cleaned and dataframe created

MD Temperature Data for 2008 cleaned and dataframe created

MD Temperature Data for 2007 cleaned and dataframe created

MD Temperature Data for 2006 cleaned and dataframe created

MD Temperature Data for 2005 cleaned and dataframe created



#### Combining dataframes for years 2005 to 2018 into one dataframe

In [27]:
Station = MDtemp2005.index
MDtemp = pd.DataFrame()
for i in range(2005,2019):
    MDtemp.reset_index(drop=True, inplace=True)
    globals()['MDtemp'+str(i)].reset_index(drop=True, inplace=True)
    MDtemp = pd.concat([MDtemp, globals()['MDtemp'+str(i)]], axis =1)
    
MDtemp = MDtemp.set_index(Station)
MDtemp = MDtemp.transpose()
MDtemp.to_csv(path_a+'\\MDtemp.csv')

print("All dataframe combined and temperature data for MD saved as .csv")

All dataframe combined and temperature data for MD saved as .csv


In [28]:
MDtemp

Station,MARYLAND,SOUTHERN EASTERN,SHORE 01,ASSATEAGUE,PRINCESS ANNE,SALISBURY,SALISBURY POLICE BRKS,SALISBURY FAA AP,SNOW HILL 4 N,--DIVISIONAL DATA------->,...,NEWARK UNIV FARM,WILMINGTON NEW CASTLE R,WILMINGTON PORTER RSVR,--DIVISIONAL DATA------->,SOUTHERN 02,DOVER,GREENWOOD 2NE,LEWES,SMYRNA 1 W,--DIVISIONAL DATA------->
Jan_Temp_05,37.000000,33.520000,37.000000,37.000000,35.000000,38.000000,37.000000,36.000000,37.000000,36.0,...,34.0,37.000000,33.520000,35.000000,33.520000,33.520000,33.520000,33.520000,33.520000,33.520000
Jan_Depart_05,1.000000,1.833333,1.833333,1.833333,1.833333,1.000000,1.000000,1.833333,1.000000,2.0,...,1.0,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333
Feb_Temp_05,38.000000,36.076923,40.000000,39.000000,36.000000,39.000000,39.000000,38.000000,39.000000,39.0,...,35.0,37.000000,36.076923,37.000000,36.076923,36.076923,36.076923,36.076923,36.076923,36.076923
Feb_Depart_05,1.571429,1.571429,1.000000,1.571429,1.000000,1.571429,1.000000,1.571429,1.000000,2.0,...,1.0,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429
Mar_Temp_05,41.000000,38.905660,43.000000,42.000000,39.000000,41.000000,41.000000,42.000000,42.000000,42.0,...,39.0,41.000000,38.905660,40.000000,38.905660,38.905660,38.905660,38.905660,38.905660,38.905660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nov_Depart_18,-1.000000,0.000000,-0.000000,-3.000000,-1.000000,-2.000000,-2.540541,-3.000000,-1.000000,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dec_Temp_18,38.615385,43.000000,42.000000,45.000000,43.000000,41.000000,41.000000,39.000000,38.615385,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dec_Depart_18,3.000000,3.000000,1.000000,2.000000,2.096774,1.000000,3.000000,2.000000,2.000000,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ann_Temp_18,48.138889,61.000000,58.000000,4.000000,58.000000,58.000000,58.000000,2.000000,48.138889,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 3

# Data Visualization

In [29]:
path_a = os.getcwd() + '\\MD\\'
path_a

'C:\\Users\\dharm\\Jupyter Workspace\\project\\MD\\'

#### Map to show the location of weather stations accross the MD state

Limitations: 
    We used the coordinates of the weather stations given inside the report by NOAA which were not completly accurate and hence the markers on the map are not accurate

In [30]:
file = path_a + 'MDrain.csv'
mdrain=pd.read_csv(file, index_col=None)

In [31]:
m = folium.Map(location=[40.0458, -76.641273],
              zoom_start=7)

df=pd.read_csv(path_a+'MDCoord.csv',index_col=None)

df_copy=df.copy()
df_copy=df_copy.drop(columns=(['County','Location']))
locList=df_copy.values.tolist()
#print(locList)
for point in range(0,len(locList)):
    folium.Marker(locList[point],popup=df['Location'][point]).add_to(m)

m

In [32]:
mdrain=mdrain.rename(columns={'Unnamed: 0':'Month and Year'})
mdrain

,Month and Year,SOUTHERN EASTERN,SHORE 01,ASSATEAGUE,PRINCESS ANNE,SALISBURY,SALISBURY POLICE BRKS,SALISBURY FAA AP,SNOW HILL 4 N,--DIVISIONAL DATA------->,...,ABERDEEN PHILLIPS FLD,CATOCTIN MOUNTAIN PARK,CONOWINGO DAM,CYLBURN,DAMASCUS 3 SSW,EMMITSBURG 2 SE,FREDERICK 2 NNE,MILLERS 4 NE,PORT DEPOSIT 2 NE,POTOMAC FILTER PLANT
0,JAN_Precip_05,4.510000,3.430000,4.88,5.04,4.04,4.19,4.370000,4.12,2.180000,...,1.520000,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556,3.590556
1,JAN_Depart_05,0.860000,0.000000,-0.05,0.17,0.37,0.01,0.000000,0.14,-0.550000,...,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250,0.131250
2,FEB_Precip_05,3.090000,2.940000,2.52,2.61,3.04,2.73,2.820000,2.25,2.100000,...,1.610000,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333,1.953333
3,FEB_Depart_05,-0.500000,0.000000,-0.93,0.00,-0.52,-0.68,-0.600000,-1.10,-1.230000,...,-0.970000,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167,-0.639167
4,MAR_Precip_05,4.600000,3.910000,4.69,2.85,4.05,4.84,4.420000,4.43,4.440000,...,5.000000,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833,4.570833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,OCT_Depart_18,3.500000,0.635385,5.18,-2.04,4.61,2.19,2.670000,4.57,0.635385,...,0.635385,0.635385,0.635385,0.635385,0.635385,0.635385,0.635385,0.635385,NaN,NaN
332,NOV_Precip_18,5.860000,6.390000,4.50,4.50,5.13,7.82,7.203226,6.89,8.330000,...,7.203226,7.203226,7.203226,7.203226,7.203226,7.203226,7.203226,7.203226,NaN,NaN
333,NOV_Depart_18,2.470000,3.734000,1.07,0.98,1.95,4.17,3.734000,3.30,3.734000,...,3.734000,3.734000,3.734000,3.734000,3.734000,3.734000,3.734000,3.734000,NaN,NaN
334,DEC_Precip_18,5.330345,4.670000,3.53,3.29,3.83,4.61,4.610000,4.92,5.330345,...,5.330345,5.330345,5.330345,5.330345,5.330345,5.330345,5.330345,5.330345,NaN,NaN


In [33]:
mdprecip = mdrain.iloc[::2]  # even
mdprecip = mdprecip[['Month and Year','NORTHERN CENTRAL   06','NORTHERN EASTERN','UPPER SOUTHERN     04','LOWER SOUTHERN     03','CENTRAL EASTERN','SOUTHERN EASTERN']]
mdprecip

,Month and Year,NORTHERN CENTRAL 06,NORTHERN EASTERN,UPPER SOUTHERN 04,LOWER SOUTHERN 03,CENTRAL EASTERN,SOUTHERN EASTERN
0,JAN_Precip_05,4.61,4.08,4.46,3.75,4.12,4.510000
2,FEB_Precip_05,1.81,2.15,1.29,1.66,2.18,3.090000
4,MAR_Precip_05,4.20,5.03,4.01,5.13,4.44,4.600000
6,APR_Precip_05,4.23,4.78,4.11,3.81,4.81,2.570000
8,MAY_Precip_05,2.98,2.30,3.27,2.64,4.78,3.480000
...,...,...,...,...,...,...,...
326,AUG_Precip_18,6.24,6.53,5.94,6.26,4.28,3.220000
328,SEP_Precip_18,9.31,11.56,10.39,11.76,8.98,7.040000
330,OCT_Precip_18,1.63,2.44,3.16,3.76,8.27,6.800000
332,NOV_Precip_18,5.83,8.15,7.91,9.06,7.61,5.860000


#### Analyzing the change in precipitation in the Northen Central Maryland from 2005-2018

In [34]:
mdprecip1 = mdprecip[['Month and Year','NORTHERN CENTRAL   06']]

mdprecip1.iplot(
    x='Month and Year',
    y='NORTHERN CENTRAL   06',
    # Specify the category
    #categories='publication',
    xTitle='Month and Year',
    yTitle='Precipitation',
    title='Precipitation in Northen Central Maryland over years 2005-2018',
    showgrid = False)

Inference: We see a normal trend across seasons in the Northen Central region of Maryland. We looked for changes in rainfall patterns, but it is difficult to differentiate with this amount of data between natural variability and variability due to climate change.

#### Comparing the average precipitation over years with different regions of Maryland

In [35]:
avg = pd.DataFrame(columns = ['Year','NORTHERN CENTRAL','NORTHERN EASTERN','UPPER SOUTHERN','LOWER SOUTHERN','CENTRAL EASTERN','SOUTHERN EASTERN'])

In [36]:
seprecip = mdprecip[['Month and Year','NORTHERN CENTRAL   06','NORTHERN EASTERN','UPPER SOUTHERN     04','LOWER SOUTHERN     03','CENTRAL EASTERN','SOUTHERN EASTERN']]
seprecip = seprecip.reset_index()
seprecip = seprecip.drop(columns='index')
avg = pd.DataFrame(columns = ['Year','NORTHERN CENTRAL','NORTHERN EASTERN','UPPER SOUTHERN','LOWER SOUTHERN','CENTRAL EASTERN','SOUTHERN EASTERN'])
avg['Year'] = [i for i in range(2005,2019)]
x = 2005
z = 0
for i in range(-1,167,12):
    y = seprecip.loc[i+1:i+12].mean()
    avg.iloc[z,1] = y[0]
    avg.iloc[z,2] = y[1]
    avg.iloc[z,3] = y[2]
    avg.iloc[z,4] = y[3]
    avg.iloc[z,5] = y[4]
    avg.iloc[z,6] = y[5]
    z = z+1

avg=avg.set_index('Year')
avg

,NORTHERN CENTRAL,NORTHERN EASTERN,UPPER SOUTHERN,LOWER SOUTHERN,CENTRAL EASTERN,SOUTHERN EASTERN
Year,,,,,,
2005,3.9025,3.70917,4.0275,4.09417,4.01833,3.27333
2006,3.33203,3.59583,3.79,4.16583,4.03583,3.36583
2007,2.50853,0.899167,3.18,2.9475,2.61,2.5475
2008,3.74432,3.98583,4.69,4.23917,3.7225,4.085
2009,4.30083,3.81,4.98333,4.95667,4.6775,4.64333
2010,3.42333,2.92917,3.46667,3.96167,3.50083,3.04
2011,4.79667,5.29,4.60917,4.64889,4.2625,3.26197
2012,3.83118,3.26291,2.84,3.32833,3.5275,3.57219
2013,3.16083,4.07,4.11083,3.41917,4.07667,3.81196


In [37]:
avg.iplot(kind='bar', xTitle='Year', yTitle='Average Precipitation',
    title='Yearly Average Precipition of Maryland over years 2005-2018', showgrid = False)

Inference: We see a huge increase in the average rainfall in all the regions of Maryland in 2018, but the factors responsible for this behaviour are unclear and still a quesiton of research.

#### Comparing precipitation in different geographical regions of Maryland 

In [38]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=mdprecip['Month and Year'], y=mdprecip['NORTHERN CENTRAL   06'], name="NORTHERN CENTRAL"))
fig3.add_trace(go.Scatter(x=mdprecip['Month and Year'], y=mdprecip['NORTHERN EASTERN'], name="NORTHERN EASTERN"))
fig3.add_trace(go.Scatter(x=mdprecip['Month and Year'], y=mdprecip['CENTRAL EASTERN'], name="Central Eastern"))
fig3.add_trace(go.Scatter(x=mdprecip['Month and Year'], y=mdprecip['UPPER SOUTHERN     04'], name="UPPER SOUTHERN"))
fig3.add_trace(go.Scatter(x=mdprecip['Month and Year'], y=mdprecip['LOWER SOUTHERN     03'], name="LOWER SOUTHERN"))
fig3.add_trace(go.Scatter(x=mdprecip['Month and Year'], y=mdprecip['SOUTHERN EASTERN'], name="SOUTHERN EASTERN"))

fig3.update_layout(title_text='Precipation of Different Regions in Maryland from 2005-2018',
                  xaxis_rangeslider_visible=True)
fig3.show(showgrid = False)

Inference: From 2005 to 2015 we see that all the regions of Maryland have received nearly same amount of rainfall, but from 2016 all the regions show a different amount of rainfall and do not behave the same way.

In [39]:
file = path_a + 'MDtemp.csv'
mdtemp=pd.read_csv(file, index_col=None)
mdtemp=mdtemp.rename(columns={'Unnamed: 0':'Month and Year'})
mdtemp

,Month and Year,MARYLAND,SOUTHERN EASTERN,SHORE 01,ASSATEAGUE,PRINCESS ANNE,SALISBURY,SALISBURY POLICE BRKS,SALISBURY FAA AP,SNOW HILL 4 N,...,NEWARK UNIV FARM,WILMINGTON NEW CASTLE R,WILMINGTON PORTER RSVR,--DIVISIONAL DATA------->.8,SOUTHERN 02,DOVER,GREENWOOD 2NE,LEWES,SMYRNA 1 W,--DIVISIONAL DATA------->.9
0,Jan_Temp_05,37.000000,33.520000,37.000000,37.000000,35.000000,38.000000,37.000000,36.000000,37.000000,...,34.0,37.000000,33.520000,35.000000,33.520000,33.520000,33.520000,33.520000,33.520000,33.520000
1,Jan_Depart_05,1.000000,1.833333,1.833333,1.833333,1.833333,1.000000,1.000000,1.833333,1.000000,...,1.0,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333,1.833333
2,Feb_Temp_05,38.000000,36.076923,40.000000,39.000000,36.000000,39.000000,39.000000,38.000000,39.000000,...,35.0,37.000000,36.076923,37.000000,36.076923,36.076923,36.076923,36.076923,36.076923,36.076923
3,Feb_Depart_05,1.571429,1.571429,1.000000,1.571429,1.000000,1.571429,1.000000,1.571429,1.000000,...,1.0,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429,1.571429
4,Mar_Temp_05,41.000000,38.905660,43.000000,42.000000,39.000000,41.000000,41.000000,42.000000,42.000000,...,39.0,41.000000,38.905660,40.000000,38.905660,38.905660,38.905660,38.905660,38.905660,38.905660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,Nov_Depart_18,-1.000000,0.000000,-0.000000,-3.000000,-1.000000,-2.000000,-2.540541,-3.000000,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,Dec_Temp_18,38.615385,43.000000,42.000000,45.000000,43.000000,41.000000,41.000000,39.000000,38.615385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,Dec_Depart_18,3.000000,3.000000,1.000000,2.000000,2.096774,1.000000,3.000000,2.000000,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,Ann_Temp_18,48.138889,61.000000,58.000000,4.000000,58.000000,58.000000,58.000000,2.000000,48.138889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
mdtem = mdtemp.iloc[::2]  # even
mdtem = mdtem[['Month and Year','NORTHERN CENTRAL   06','NORTHERN EASTERN','UPPER SOUTHERN     04','LOWER SOUTHERN     03','CENTRAL EASTERN','SOUTHERN EASTERN']]
mdtem

,Month and Year,NORTHERN CENTRAL 06,NORTHERN EASTERN,UPPER SOUTHERN 04,LOWER SOUTHERN 03,CENTRAL EASTERN,SOUTHERN EASTERN
0,Jan_Temp_05,33.520000,32.0,35.0,33.0,34.000000,33.520000
2,Feb_Temp_05,36.076923,34.0,39.0,37.0,36.000000,36.076923
4,Mar_Temp_05,38.905660,38.0,42.0,40.0,40.000000,38.905660
6,Apr_Temp_05,54.132075,55.0,56.0,54.0,54.000000,54.132075
8,May_Temp_05,58.096154,58.0,61.0,59.0,57.000000,58.096154
...,...,...,...,...,...,...,...
354,Sep_Temp_18,72.358974,71.0,75.0,75.0,78.000000,76.000000
356,Oct_Temp_18,58.450000,57.0,61.0,61.0,64.000000,62.000000
358,Nov_Temp_18,42.850000,40.0,45.0,45.0,47.000000,47.000000
360,Dec_Temp_18,38.615385,35.0,40.0,41.0,38.615385,43.000000


#### Analyzing the change in temperature in the Northen Central of Maryland from 2005-2018

In [41]:
year_nc = pd.read_csv(path_a+"Year_NC.csv")
year_nc

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,33.520000,40.478261,36.510204,34.978261,28.217391,31.476190,28.418605,36.000000,33,22,29.921053,31.236842,37.461538,30.342105
1,36.076923,35.531915,27.604167,36.437500,36.369565,29.977778,36.181818,39.857143,32,28,24.550000,35.025641,42.714286,40.589744
2,38.905660,43.510204,44.204545,43.760870,42.326087,46.813953,41.744186,49.000000,38,35,37.125000,48.775000,41.692308,38.205128
3,54.132075,55.875000,50.173913,54.787234,53.978261,56.760870,55.700000,50.000000,53,50,53.589744,52.275000,58.736842,50.128205
4,58.096154,61.693878,64.159091,59.422222,62.851064,65.066667,65.095238,66.000000,60,61,65.307692,60.170732,61.125000,68.000000
5,72.627451,71.108696,72.369565,73.723404,70.250000,75.630435,71.384615,68.000000,72,70,72.375000,71.641026,71.650000,71.410256
6,77.428571,77.304348,74.978723,75.744681,72.936170,78.555556,77.076923,76.000000,76,72,75.121951,77.500000,76.763158,75.947368
7,77.058824,76.687500,76.066667,72.127660,75.750000,75.931818,74.292683,73.000000,71,69,73.666667,77.225000,72.128205,76.775000
8,71.039216,65.040816,69.521739,68.361702,66.102041,69.760870,66.761905,65.000000,65,64,70.461538,70.894737,67.948718,72.358974
9,57.380000,54.638298,62.739130,54.466667,54.812500,57.162791,55.000000,54.000000,57,54,55.000000,58.500000,60.473684,58.450000


In [42]:
fig = go.Figure()
fig.add_trace(go.Box(y=year_nc['2006'], name="2006"))
fig.add_trace(go.Box(y=year_nc['2007'], name="2007"))
fig.add_trace(go.Box(y=year_nc['2008'], name="2008"))
fig.add_trace(go.Box(y=year_nc['2009'], name="2009"))
fig.add_trace(go.Box(y=year_nc['2010'], name="2010"))
fig.add_trace(go.Box(y=year_nc['2011'], name="2011"))
fig.add_trace(go.Box(y=year_nc['2012'], name="2012"))
fig.add_trace(go.Box(y=year_nc['2013'], name="2013"))
fig.add_trace(go.Box(y=year_nc['2014'], name="2014"))
fig.add_trace(go.Box(y=year_nc['2015'], name="2015"))
fig.add_trace(go.Box(y=year_nc['2016'], name="2016"))
fig.add_trace(go.Box(y=year_nc['2017'], name="2017"))
fig.add_trace(go.Box(y=year_nc['2018'], name="2018"))
fig.update_layout(
    title='Analysing temperature statistics of Northern Central Maryland')
fig.show()

Inference: We can see that the maximum temperature of Northern Central Maryland is around the same over the years but the minimum temperature has more variation.

#### Comparing the deviation in average temperature over every year and for different regions of Maryland

In [43]:
avg = pd.DataFrame(columns = ['Year','NORTHERN CENTRAL','NORTHERN EASTERN','UPPER SOUTHERN','LOWER SOUTHERN','CENTRAL EASTERN','SOUTHERN EASTERN'])

In [44]:
setemp = mdtemp[['Month and Year','NORTHERN CENTRAL   06','NORTHERN EASTERN','UPPER SOUTHERN     04','LOWER SOUTHERN     03','CENTRAL EASTERN','SOUTHERN EASTERN']]
setemp = setemp.reset_index()
setemp = setemp.drop(columns='index')
avg = pd.DataFrame(columns = ['Year','NORTHERN CENTRAL','NORTHERN EASTERN','UPPER SOUTHERN','LOWER SOUTHERN','CENTRAL EASTERN','SOUTHERN EASTERN'])
avg['Year'] = [i for i in range(2005,2019)]
x = 2005
z = 0
for i in range(-1,167,12):
    y = setemp.loc[i+1:i+12].mean()
    avg.iloc[z,1] = y[0]
    avg.iloc[z,2] = y[1]
    avg.iloc[z,3] = y[2]
    avg.iloc[z,4] = y[3]
    avg.iloc[z,5] = y[4]
    avg.iloc[z,6] = y[5]
    z = z+1

avg=avg.set_index('Year')
avg

,NORTHERN CENTRAL,NORTHERN EASTERN,UPPER SOUTHERN,LOWER SOUTHERN,CENTRAL EASTERN,SOUTHERN EASTERN
Year,,,,,,
2005,25.6113,25.131,26.5289,25.5833,25.796,25.6113
2006,31.7055,31.7055,32.6806,31.4167,31.0868,31.7055
2007,25.718,25.1272,25.9203,25.8389,26.8845,25.718
2008,33.7791,33.7791,34.0632,33.5765,35.723,33.7791
2009,23.3389,23.2741,24.4925,23.3398,24.7676,23.3389
2010,36.3394,36.2698,37.7698,36.3394,37.4981,36.3394
2011,21.9563,20.6456,22.7385,22.2148,22.7053,22.408
2012,34.6213,32.774,35.1265,35.5926,35.4528,34.4053
2013,22.2347,20.6011,22.4167,22.9545,23.8898,22.3378


#### Comparing the deviation in temperature across different regions of MD in consecutive years.

In [45]:
avg_diff = avg.diff().T
avg_diff = avg_diff.drop([2005], axis=1)
avg_diff

Year,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
NORTHERN CENTRAL,6.09417,-5.98747,8.06112,-10.4402,13.0005,-14.383,12.6649,-12.3866,10.0839,-7.21122,6.13701,-2.17215,-3.28542
NORTHERN EASTERN,6.57451,-6.57824,8.65189,-10.505,12.9957,-15.6242,12.1285,-12.173,10.8525,-7.547,6.25058,-1.94048,-10.0829
UPPER SOUTHERN,6.15173,-6.76028,8.14293,-9.57076,13.2773,-15.0313,12.388,-12.7099,10.1667,-7.2851,6.70177,-2.36667,-2.46667
LOWER SOUTHERN,5.83333,-5.57778,7.73758,-10.2367,12.9996,-14.1246,13.3778,-12.6381,10.4372,-7.53791,6.47002,-2.51465,-3.05916
CENTRAL EASTERN,5.29075,-4.20227,8.83852,-10.9554,12.7305,-14.7927,12.7475,-11.5631,9.78502,-6.88149,5.89343,-1.65038,-3.95301
SOUTHERN EASTERN,6.09417,-5.98747,8.06112,-10.4402,13.0005,-13.9314,11.9973,-12.0674,9.98209,-6.81801,6.51098,-2.64556,-2.97928


In [46]:
fig = go.Figure(data=go.Heatmap(
        z=avg_diff,
        y=avg_diff.index,
        x=avg_diff.columns,
        colorscale='Magma'))

fig.update_layout(
    title='Analysing deviation in temperature across different regions of MD')

fig.show()

Inference: Here we see alternative rise and fall of temperature over the years from 2006, and we also note that the difference is nearly similar accross all the regions in MD

#### Comparing temperature in different geographical regions of Maryland 

In [47]:
fig4 = go.Figure()

fig4.add_trace(go.Scatter(x=mdtem['Month and Year'], y=mdtem['NORTHERN CENTRAL   06'], name="NORTHERN CENTRAL"))
fig4.add_trace(go.Scatter(x=mdtem['Month and Year'], y=mdtem['NORTHERN EASTERN'], name="NORTHERN EASTERN"))
fig4.add_trace(go.Scatter(x=mdtem['Month and Year'], y=mdtem['CENTRAL EASTERN'], name="Central Eastern"))
fig4.add_trace(go.Scatter(x=mdtem['Month and Year'], y=mdtem['UPPER SOUTHERN     04'], name="UPPER SOUTHERN"))
#fig4.add_trace(go.Scatter(x=mdtem['Month and Year'], y=mdtem['LOWER SOUTHERN     04'], name="Central Eastern"))
fig4.add_trace(go.Scatter(x=mdtem['Month and Year'], y=mdtem['SOUTHERN EASTERN'], name="SOUTHERN EASTERN"))

fig4.update_layout(title_text='Temperature of Different Regions in Maryland from 2005-2018',
                  xaxis_rangeslider_visible=True)
fig4.show()

Inference: According to NASA "The planet's average surface temperature has risen about 1.62 degrees Fahrenheit (0.9 degrees Celsius) since 1880", hence we cannot really see a change in temperature for a small state over 10 years of data. But the temperature behaves in a similar pattern over each year.

Limitation: As mentioned above, this graph does not help us prove anything as we are limited by the scope of data. But if we had have been lucky and got text based pdf instead of image based pdf for 119 years then we would have been able to find around 1 degree Fahrenheit increase over the years.

PS: Due to the complexity of data cleaning, pre processing for different visualization and time constraints we had to leave the Virginia data aside.

# Conclusion

This project has been a huge learning curve for us. We learnt to deal with really unclean data and also how limited data can drastically decrease the scope of the project. 
Our findings indicate that precipitation can have a much more visible change over a short period than temperature. Also, it is tough to conclude that the few changes we did observe were due to climate change and not due to any other factor. 
To get more concrete results and conclusions, a much larger dataset spanning a much greater number of years is needed. But if we can observe changes even in such a small dataset, imagine what we would have found with 119 years of data.